In [ ]:
import cv2
import numpy as np
import mediapipe as mp

# Initialize MediaPipe
mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(max_num_hands=1)

# Setup
cap = cv2.VideoCapture(0)
canvas = None
colors = [(0, 165, 255), (0, 255, 0), (0, 0, 255)]  # Orange, Green, Red
colorIndex = 0
brushThickness = 5
xp, yp = 0, 0

# Button dimensions
btn_w, btn_h = 40, 40
gap = 10

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.flip(frame, 1)
    h, w, _ = frame.shape

    if canvas is None:
        canvas = np.zeros_like(frame)

    # Convert color and process with MediaPipe
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    result = hands.process(rgb)

    # Draw color selection buttons
    for i, col in enumerate(colors):
        x = i * (btn_w + gap)
        y = 10
        cv2.rectangle(frame, (x, y), (x + btn_w, y + btn_h), col, -1)

    # Brush size buttons
    brush_labels = ["S", "M", "L"]
    for i, label in enumerate(brush_labels):
        x = 3 * (btn_w + gap) + i * (btn_w + gap)
        y = 10
        cv2.rectangle(frame, (x, y), (x + btn_w, y + btn_h), (200, 200, 200), -1)
        cv2.putText(frame, label, (x + 10, y + 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 0), 2)

    # Clear button
    clear_x = 6 * (btn_w + gap)
    clear_y = 10
    cv2.rectangle(frame, (clear_x, clear_y), (clear_x + 80, clear_y + btn_h), (0, 0, 0), -1)
    cv2.putText(frame, "CLEAR", (clear_x + 5, clear_y + 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)

    # Hand tracking
    if result.multi_hand_landmarks:
        for handLms in result.multi_hand_landmarks:
            lmList = []
            for id, lm in enumerate(handLms.landmark):
                lmList.append((int(lm.x * w), int(lm.y * h)))

            if len(lmList) >= 8:
                x1, y1 = lmList[8]  # Index fingertip

                cv2.circle(frame, (x1, y1), 8, (0, 0, 0), -1)

                # Finger detection
                fingers = []
                fingers.append(1 if lmList[4][0] < lmList[3][0] else 0)   # Thumb
                fingers.append(1 if lmList[8][1] < lmList[6][1] else 0)   # Index
                fingers.append(1 if lmList[12][1] < lmList[10][1] else 0) # Middle
                fingers.append(1 if lmList[16][1] < lmList[14][1] else 0) # Ring
                fingers.append(1 if lmList[20][1] < lmList[18][1] else 0) # Pinky

                # Selection Mode
                if fingers[1] == 1 and sum(fingers) == 1 and y1 < 60:
                    xp, yp = 0, 0
                    if 0 < x1 < btn_w:
                        colorIndex = 0
                    elif btn_w + gap < x1 < 2 * (btn_w + gap):
                        colorIndex = 1
                    elif 2 * (btn_w + gap) < x1 < 3 * (btn_w + gap):
                        colorIndex = 2
                    elif 3 * (btn_w + gap) < x1 < 4 * (btn_w + gap):
                        brushThickness = 5
                    elif 4 * (btn_w + gap) < x1 < 5 * (btn_w + gap):
                        brushThickness = 15
                    elif 5 * (btn_w + gap) < x1 < 6 * (btn_w + gap):
                        brushThickness = 30
                    elif 6 * (btn_w + gap) < x1 < 6 * (btn_w + gap) + 80:
                        canvas = np.zeros_like(frame)

                # Drawing Mode
                elif fingers[1] == 1 and sum(fingers) == 1:
                    if xp == 0 and yp == 0:
                        xp, yp = x1, y1
                    cv2.line(frame, (xp, yp), (x1, y1), colors[colorIndex], brushThickness)
                    cv2.line(canvas, (xp, yp), (x1, y1), colors[colorIndex], brushThickness)
                    xp, yp = x1, y1

            mp_drawing.draw_landmarks(frame, handLms, mp_hands.HAND_CONNECTIONS)

    # Merge canvas and frame
    gray = cv2.cvtColor(canvas, cv2.COLOR_BGR2GRAY)
    _, inv = cv2.threshold(gray, 50, 255, cv2.THRESH_BINARY_INV)
    inv = cv2.cvtColor(inv, cv2.COLOR_GRAY2BGR)
    frame = cv2.bitwise_and(frame, inv)
    frame = cv2.bitwise_or(frame, canvas)

    cv2.imshow("Virtual Paint", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

   